In [41]:
import pandas as pd

### Assign weight per each model
This data get from individual f1-score per each model.

In [42]:
lstm_weight = {
    'B-AGE': 71.32,
    'B-HOSPITAL': 75.22,
    'B-MEDICALRECORD': 65.37,
    'B-ORGANIZATION': 21.88,
    'B-PER': 66.52,
    'B-PROFESSION': 39.44,
    'B-STATE': 71.43,
    'B-STREET': 20.41,
    'CITY': 62.82,
    'COUNTRY': 12.1,
    'DATE': 79.13,
    'EMAIL': 0.01,
    'FAX': 0.03,
    'I-AGE': 0.28,
    'I-HOSPITAL': 76.37,
    'I-MEDICALRECORD': 45.54,
    'I-ORGANIZATION': 8.12,
    'I-PER': 49.86,
    'I-PROFESSION': 45.57,
    'I-STATE': 0.04,
    'I-STREET': 75.82,
    'IDNUM': 46.5,
    'O': 100,
    'PHONE': 24.2,
    'USERNAME': 50.89,
    'ZIP': 60.38
}


In [43]:
spacy_weight = {
    'B-AGE': 75.94,
    'B-HOSPITAL': 71.71,
    'B-MEDICALRECORD': 72.16,
    'B-ORGANIZATION': 40.74,
    'B-PER': 67.86,
    'B-PROFESSION': 41.79,
    'B-STATE': 80.84,
    'B-STREET': 15.87,
    'CITY': 54.39,
    'COUNTRY': 48.57,
    'DATE': 88.6,
    'EMAIL': 66.67,
    'FAX': 0,
    'I-AGE': 0,
    'I-HOSPITAL': 77.58,
    'I-MEDICALRECORD': 34.36,
    'I-ORGANIZATION': 25.88,
    'I-PER': 52.59,
    'I-PROFESSION': 58.09,
    'I-STATE': 0,
    'I-STREET': 76.71,
    'IDNUM': 58.02,
    'O': 98.43,
    'PHONE': 42.72,
    'USERNAME': 92.94,
    'ZIP': 56.27
}


In [44]:
bert_weight = {
    'B-AGE': 0.51,
    'B-HOSPITAL': 0,
    'B-MEDICALRECORD': 0,
    'B-ORGANIZATION': 0,
    'B-PER': 31.93,
    'B-PROFESSION': 0,
    'B-STATE': 5.69,
    'B-STREET': 0,
    'CITY': 0,
    'COUNTRY': 91.54,
    'DATE': 0,
    'EMAIL': 0,
    'FAX': 60.4,
    'I-AGE': 0,
    'I-HOSPITAL': 48.97,
    'I-MEDICALRECORD': 0,
    'I-ORGANIZATION': 11.06,
    'I-PER': 69.61,
    'I-PROFESSION': 8.33,
    'I-STATE': 69.19,
    'I-STREET': 0,
    'IDNUM': 38.82,
    'O': 98.56,
    'PHONE': 32.67,
    'USERNAME': 28.03,
    'ZIP': 0
}


### load data that is compbination of three model out put over the test file

In [45]:
test_data = pd.read_csv("/home/jovyan/work/data/hybrid/spacy_output.csv", low_memory=False)


### Voting system

In [58]:
import pandas as pd

def apply_voting_system(df, lstm_col, spacy_col, bert_col):
    # Apply voting system
    votes = []

    for lstm_tag, spacy_tag, bert_tag in zip(df[lstm_col], df[spacy_col], df[bert_col]):
        if lstm_tag == spacy_tag == bert_tag:
            votes.append(lstm_tag)
        else:
            tag_counts = count_occurrences([lstm_tag,spacy_tag,bert_tag])
            votes.append(vote_court(tag_counts))
    return votes


In [59]:
def count_occurrences(lst):
    counts = {}
    for tag in lst:
        counts[tag] = lst.count(tag)
    return counts

In [60]:
def vote_court(tag_counts):
    tag_counts =tag_counts
    if "O" in tag_counts.keys():
        del tag_counts["O"]

    most_common_tag = max(tag_counts, key=tag_counts.get)
    return most_common_tag


In [61]:
# Mock-up data
data = {
    "lstm": ["O", "O", "tag2", "tag2"],
    "spacy": ["tag1", "O", "tag3", "tag2"],
    "bert": ["O", "O", "O", "O"]
}

voting_col = apply_voting_system(data,"lstm","spacy","bert")

In [62]:
voting_col

['tag1', 'O', 'tag2', 'tag2']

### Arg max

In [55]:
def apply_arg_max(df, lstm_col, spacy_col, bert_col):
    # Apply voting system
    arg_max_col = []

    for lstm_tag, spacy_tag, bert_tag in zip(df[lstm_col], df[spacy_col], df[bert_col]):
        if lstm_tag == spacy_tag == bert_tag:
            voting_col.append(lstm_tag)
        else:
            arg_max_col.append(arg_max_court(lstm_tag, spacy_tag, bert_tag))
    return voting_col


In [56]:
def arg_max_court(lstm_tag, spacy_tag, bert_tag):
    print(lstm_tag, spacy_tag, bert_tag)
    print(lstm_weight[lstm_tag],spacy_weight[spacy_tag],bert_weight[bert_tag])
    tags = [lstm_tag, spacy_tag, bert_tag]
    weight = [lstm_weight[lstm_tag],spacy_weight[spacy_tag],bert_weight[bert_tag]]

    for index,tag in enumerate(tags):
        if tag == "O":
            del tags[index]
            del weight[index]

    max_index = weight.index(max(weight))
    chosen_tag = tags[max_index]
    print(chosen_tag)
    return chosen_tag



In [57]:
data = {
    "lstm": ["O",     "O",  "B-PER", "COUNTRY"],
    "spacy": ["B-PER", "O", "COUNTRY", "COUNTRY"],
    "bert": ["O",      "O",  "O",      "O"]
}

voting_col = apply_arg_max(data,"lstm","spacy","bert")

O B-PER O
100 67.86 98.56
B-PER
B-PER COUNTRY O
66.52 48.57 98.56
B-PER
COUNTRY COUNTRY O
12.1 48.57 98.56
COUNTRY
